<a href="https://colab.research.google.com/github/patricio-tech/proyectos/blob/master/RL4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Monte Carlo Methods**

**Exploration-Explanation Dilemma**

In [1]:
import sys
import gym
import numpy as np
from collections import defaultdict
env = gym.make('Blackjack-v0')
print(env.observation_space)
print(env.action_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))
Discrete(2)


In [9]:
state = env.reset()
while True:
       print('state:', state)
       action = env.action_space.sample()
       print('action: ', action) 
       state, reward, done, info = env.step(action)
       if done:
          if reward > 0: 
             print('Reward: ', reward)                 
          else: 
             print('Reward: ', reward)
          break

state: (20, 9, False)
action:  0
Reward:  1.0


In [10]:
def generate_episode(env):
    episode = []
    state = env.reset()
    while True:
        probs = [0.75, 0.25] if state[0] > 18 else [0.25, 0.75]
        action = np.random.choice(np.arange(2), p=probs)
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

In [11]:
for i in range(10):
    print(generate_episode(env))

[((16, 8, True), 1, 0.0), ((16, 8, False), 1, -1.0)]
[((21, 10, True), 0, 1.0)]
[((14, 3, False), 0, -1.0)]
[((15, 2, False), 1, 0.0), ((17, 2, False), 1, 0.0), ((21, 2, False), 0, 1.0)]
[((17, 10, False), 0, 1.0)]
[((17, 10, True), 0, -1.0)]
[((15, 3, True), 1, 0.0), ((17, 3, True), 1, 0.0), ((19, 3, True), 0, 1.0)]
[((15, 10, True), 1, 0.0), ((16, 10, True), 0, -1.0)]
[((15, 1, False), 0, -1.0)]
[((20, 7, False), 0, -1.0)]
